## The following code supposed to demonstrate how it easy to replace a Java docx4j wall of code with tiny DSL of DocxKtm

Suppose we want to create docx with several paragraphs with random text, footer with page numbers anf a table with some rows and cells.
How it would look in pure docx4j:

In [ ]:
fun main() {
    val wordMLPackage = WordprocessingMLPackage.createPackage()
    val mainDocumentPart = wordMLPackage.mainDocumentPart
    val factory = Context.getWmlObjectFactory()

    // --- Footer with page numbers ---
    val footerPart = FooterPart(PartName("/word/footer1.xml"))
    val footer = factory.createFooter()

    val p = factory.createP()
    val r = factory.createR()
    val fldCharBegin = factory.createFldChar().apply { fldCharType = STFldCharType.BEGIN }
    val fldCharSeparate = factory.createFldChar().apply { fldCharType = STFldCharType.SEPARATE }
    val fldCharEnd = factory.createFldChar().apply { fldCharType = STFldCharType.END }

    fun createInstrText(value: String): R {
        val run = factory.createR()
        val instr = factory.createText().apply {
            this.value = value
            this.space = "preserve"
        }
        run.content.add(factory.createRInstrText(instr))
        return run
    }

    r.content.add(factory.createRFldChar(fldCharBegin))
    r.content.add(createInstrText(" PAGE "))
    r.content.add(factory.createRFldChar(fldCharSeparate))
    r.content.add(factory.createRFldChar(fldCharEnd))

    val r2 = factory.createR()
    r2.content.add(factory.createText(" of "))

    val r3 = factory.createR()
    r3.content.add(factory.createRFldChar(factory.createFldChar().apply { fldCharType = STFldCharType.BEGIN }))
    r3.content.add(createInstrText(" NUMPAGES "))
    r3.content.add(factory.createRFldChar(factory.createFldChar().apply { fldCharType = STFldCharType.SEPARATE }))
    r3.content.add(factory.createRFldChar(factory.createFldChar().apply { fldCharType = STFldCharType.END }))

    p.content.add(r)
    p.content.add(r2)
    p.content.add(r3)

    // Center paragraph
    val pPr = factory.createPPr()
    val jc = factory.createJc()
    jc.`val` = JcEnumeration.CENTER
    pPr.jc = jc
    p.pPr = pPr

    footer.content.add(p)
    footerPart.jaxbElement = footer
    wordMLPackage.mainDocumentPart.addTargetPart(footerPart)

    val sectPr = factory.createSectPr()
    val footerRef = factory.createFooterReference()
    footerRef.id = footerPart.relationshipsPart.relationships[0].id
    footerRef.type = HdrFtrRef.DEFAULT
    sectPr.footerReference.add(footerRef)
    mainDocumentPart.document.body.sectPr = sectPr

    // --- Random paragraphs ---
    repeat(5) {
        val para = factory.createP()
        val run = factory.createR()
        val text = factory.createText()
        text.value = "This is random text paragraph #${it + 1}. Lorem ipsum dolor sit amet."
        run.content.add(text)
        para.content.add(run)
        mainDocumentPart.addObject(para)
    }

    // --- Table with random numbers ---
    val tbl = factory.createTbl()
    val borders = factory.createCTTblBorders().apply {
        top = factory.createCTBorder().apply { `val` = STBorder.SINGLE }
        bottom = factory.createCTBorder().apply { `val` = STBorder.SINGLE }
        left = factory.createCTBorder().apply { `val` = STBorder.SINGLE }
        right = factory.createCTBorder().apply { `val` = STBorder.SINGLE }
        insideH = factory.createCTBorder().apply { `val` = STBorder.SINGLE }
        insideV = factory.createCTBorder().apply { `val` = STBorder.SINGLE }
    }
    val tblPr = factory.createTblPr()
    tblPr.tblBorders = borders
    tbl.tblPr = tblPr

    repeat(3) {
        val tr = factory.createTr()
        repeat(5) {
            val tc = factory.createTc()
            val para = factory.createP()
            val run = factory.createR()
            val text = factory.createText()
            text.value = Random.nextInt(1, 100).toString()
            run.content.add(text)
            para.content.add(run)
            tc.content.add(para)
            tr.content.add(tc)
        }
        tbl.content.add(tr)
    }
    mainDocumentPart.addObject(tbl)

    // --- Save the file ---
    wordMLPackage.save(File("example.docx"))
}


And that is how we replace this with DocxKtm DSL:

In [1]:
@file:DependsOn("io.github.alexmaryin:docxktm:1.3.1")

In [6]:
import io.github.alexmaryin.docxktm.DocxNew
import io.github.alexmaryin.docxktm.extensions.body
import io.github.alexmaryin.docxktm.extensions.footer
import io.github.alexmaryin.docxktm.models.ParagraphStyle
import io.github.alexmaryin.docxktm.models.TextStyle
import io.github.alexmaryin.docxktm.parts.paragraph
import io.github.alexmaryin.docxktm.parts.table
import io.github.alexmaryin.docxktm.parts.text
import io.github.alexmaryin.docxktm.values.Alignment
import java.io.File
import kotlin.random.Random
import kotlin.random.nextInt

DocxNew("sample.docx") {
    body {
        repeat(5) { num ->
            paragraph { text("This is random text paragraph #${num + 1}. Lorem ipsum dolor sit amet.") }
        }
        table {
            repeat(3) {
                row {
                    repeat(5) {
                        textInCell(Random.nextInt(100).toString())
                    }
                }
            }
        }
    }
    footer(ParagraphStyle(alignment = Alignment.CENTER)) {
        pageNumber("page #p of #t")
    }
}

/Users/java73/IdeaProjects/docxktm//sample.docx
